In [ ]:
# Setting up Env.

import sys
!{sys.executable} -m pip install pandas-profiling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

In [ ]:
# Importing
# Modified pathing for Kaggle upload

daily_act = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv')
daily_cal = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyCalories_merged.csv')
daily_int = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyIntensities_merged.csv')
daily_stp = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailySteps_merged.csv')
daily_slp = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv')
hourly_stp = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv')
hourly_cal = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlyCalories_merged.csv')
hourly_int = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlyIntensities_merged.csv')
minutes_slp = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/minuteSleep_merged.csv')

In [ ]:
daily_act # contains all but daily_slp (start looking here)
daily_cal # in act
daily_int # in act
daily_stp # in act
daily_slp # only 413 records, act has 940
hourly_stp 
hourly_cal # doesn't add as much as the other 2, stp and int
hourly_int
minutes_slp.head()

In [ ]:
np.round(daily_act.describe(),2)

In [ ]:
np.round(hourly_stp.describe(),2)

In [ ]:
np.round(daily_slp.describe(),2)

In [ ]:
np.round(hourly_cal.describe(),2)

In [ ]:
np.round(hourly_int.describe(),2)

In [ ]:
np.round(minutes_slp.describe(),2)

In [ ]:
hourly_int.head(24)

In [ ]:
hourly_stp.head(24)

In [ ]:
# daily_act
da = daily_act
da

In [ ]:
da.dtypes

In [ ]:
da['Date'] = pd.to_datetime(da['ActivityDate'])
da.dtypes

In [ ]:
da.head()

In [ ]:
da.columns

In [ ]:
cols = list(da.columns.values)
datest = da[cols[0:2] + [cols[-1]] + cols[2:15]]
da1 = datest

In [ ]:
# Checking for duplicates da
da1.duplicated().sum()

In [ ]:
# Checking for missing data da
sns.heatmap(da1.isnull(),cbar=False)

In [ ]:
# sns.pairplot(da1)

In [ ]:
# use the picture to check out some of the trends, not enough memory in chrome to open in a new tab..

In [ ]:
# joining daily sleep with daily activity, need to make a common key
# das = daily sleep and daily activity

In [ ]:
da.head()

In [ ]:
dsl = daily_slp
dsl.head()

In [ ]:
dsl.describe()

In [ ]:
# Checking for duplicates da
dsl.duplicated().sum()

In [ ]:
dsl[dsl.duplicated()==True]
dsl.drop_duplicates(inplace=True)

In [ ]:
# duplicates gone
dsl.duplicated().sum()

In [ ]:
# finding nas
dsl.isna().sum()
# no nas

In [ ]:
dsl.dtypes

In [ ]:
# finding possible naps to drop to clean up conclusions
naps = dsl.loc[dsl['TotalMinutesAsleep'] < 120]
naps

In [ ]:
# Learning more about the 796 minutes of sleep day
longslps = dsl.loc[dsl['TotalMinutesAsleep'] > 720]
longslp = longslps.loc[longslps['TotalSleepRecords'] < 2]
longslp

In [ ]:
# making a test set and dropping extreme outliers
dsl_test=dsl
nap_longslp = [25, 27, 30, 65, 110, 123, 146, 150, 164, 172, 181, 196, 198, 282, 337,
        338,365]
dsl_result = dsl_test.drop(nap_longslp)
dsl_result.describe()

In [ ]:
# dsl1 is has had sleep totals of <120 removed and single instances of sleep >720 removed
dsl1 = dsl_result
# joining dsl1 to da by making a key
# what should the key be

In [ ]:
dsl1.dtypes

In [ ]:
da.dtypes

In [ ]:
dsl1.head(3)


In [ ]:
da.head(3)

In [ ]:
# because the dates and the ids repeat, I will join those two together to make a unique key to join the data sets
# dropped the time off the sleepday (they were all 12:00 am)
dsl1['SleepDay'] = dsl1['SleepDay'].str.split(" ", n=1, expand=True)


In [ ]:
# making a key as KeyIdDate for da
da["Key"] = da["Id"].astype(str) + da["ActivityDate"].astype(str)
da

In [ ]:
# making a key as KeyIdDate for dsl1
dsl1["Key"] = dsl1["Id"].astype(str) + dsl1["SleepDay"].astype(str)
dsl1

In [ ]:
datest = da
dsltest = dsl1

In [ ]:
# joining da and dsl1
daslp_Key = pd.merge(datest, dsltest, on=['Key'], how='inner')
daslp_Key

In [ ]:
# dropping redundant columns (ActivityDate, Key, SleepDay, Id_y)
daslp_drop = ['ActivityDate', 'Key', 'SleepDay', 'Id_y']
daslp = daslp_Key.drop(columns=daslp_drop, axis = 1)

In [ ]:
# now I want to graph the sleep data against the other data in a pair plot
sns.pairplot(daslp)


In [ ]:
daslp.columns

In [ ]:
daslp.describe()

In [ ]:
sns.pairplot(daslp, y_vars=['TotalSleepRecords', 'TotalMinutesAsleep', 'TotalTimeInBed'], kind="reg")

In [ ]:
plt.figure(figsize = (16,5))
hmdaslp = np.round(daslp.corr(),2)
sns.heatmap(hmdaslp, annot=True, linewidth = .2, cmap='PiYG', center=0, cbar=False)

In [ ]:
sns.pairplot(daslp, 
             x_vars=['ModeratelyActiveDistance', 'FairlyActiveMinutes', 'SedentaryMinutes'], 
             y_vars=['TotalSleepRecords', 'TotalMinutesAsleep', 'TotalTimeInBed'],
             kind="reg")

In [ ]:
# sedentary minutes seem to have a strong relatioship to time asleep and time in bed
# will run some ananlysis on the relationship
sns.regplot(x=daslp['SedentaryMinutes'], y=daslp['TotalMinutesAsleep'])
plt.xlabel('Sedentary Minutes')
plt.ylabel('Total Minutes Asleep')
plt.title('Total Sleep vs. Sedentary Minutes')


In [ ]:
# checking coorelation
np.round(np.corrcoef(daslp['SedentaryMinutes'],daslp['TotalMinutesAsleep'])[1][0],2)

In [ ]:
# exporting daslp for work in Tableau
daslp.to_csv('Daslp_083821.csv', index=False)

In [ ]:
# colors pulled from bellabeat website: #6c62ff  #ffc6ba  #f7f7f7
